In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/code/phasor_julia`


In [2]:
include("module.jl")
using .PhasorNetworks

┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_potential :: Tuple{Real, AbstractVector, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.potential_to_phase :: Tuple{AbstractArray, Real, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_train :: Tuple{AbstractArray, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_potential :: Tuple{Real, AbstractVector, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243


┌ Warning: Replacing docs for `Main.PhasorNetworks.potential_to_phase :: Tuple{AbstractArray, Real, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_train :: Tuple{AbstractArray, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_potential :: Tuple{Real, AbstractVector, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.potential_to_phase :: Tuple{AbstractArray, Real, Real, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.PhasorNetworks.phase_to_train :: Tuple{AbstractArray, Main.PhasorNetworks.SpikingArgs}` in module `Main.PhasorNetworks`
└ @ Base.Docs docs/Docs.jl:243
